#HW3 - SQL
 
This homework has you working with a new database of information on ticket sales for various types of events.  Your job will be to do some initial exploring and then demonstrate your ability to do all the different types of SQL queries we learned over the last two week.  You'll also need to make one function that'll make looking at the tables easier. 
 
These questions are written in the way someone would ask them to you.  In other words, I'm using 'plain english' questions vs. ones where I'm very explicit in terms of what columns and tables to use.  Your exploring of the database and functions to ease that process will come in handy here!  
 
Also, I made the database using a set of data from Amazon. You can read more about what each table contains here: https://docs.aws.amazon.com/redshift/latest/dg/c_sampledb.html.  

## Libraries and import functions

First bring the libraries we'll need!

In [2]:
import psycopg2 
import pandas as pd
import numpy as np

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Now bring in all our functions we used in the lessons!  

In [3]:
# Make our connection/cursor function 

def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = psycopg2.connect(
    host="ticket-sales-db.cnt7cm8tgir1.us-west-1.rds.amazonaws.com",
    database="sales_db",
    user="postgres",
    password="ISTA322SALESDB",
    port='5432')
  
  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor

# Same run_query function
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store

  cur.close() # close
  conn.close() # close

  return(my_data) # return

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results

## Make a SQL head function - 1 point

Make function to get the pandas equivalent of `.head()`

This function should be called `sql_head` and take a single argument of `table_name` where you specify the table name you want the head information from.  It should return the column names along with the first five rows of the table along.  

High-five if you return a pandas dataframe with this information so it displays nicely:)

In [4]:
# make sql_head function
def sql_head(table_name):
   new =  run_query(f'SELECT * FROM {table_name} LIMIT 5')
   new = list(new)
   for i in range(len(new[1])):
     new[1][i] = list((new[1][i]))
   return pd.DataFrame(data = np.array(new[1]), columns = new[0])

In [5]:
# Check that it works!
sql_head(table_name = 'event')

,event_id,venue_id,cat_id,date_id,event_name,start_time
0,2,306,8,2114,Boris Godunov,2008-10-15 20:00:00
1,3,302,8,1935,Salome,2008-04-19 14:30:00
2,4,309,8,2090,La Cenerentola (Cinderella),2008-09-21 14:30:00
3,5,302,8,1982,Il Trovatore,2008-06-05 19:00:00
4,6,308,8,2109,L Elisir d Amore,2008-10-10 19:30:00


## Explore and SELECT - 1 point

Let's start this homework with some basic queries to get a look at what's in the various tables.  I want you to do the following.

* Use your get_table_names() function to see what tables are in the database.
* Use your get_column_names() to get the column names of each of the tables.  **Do this all within a single cell to keep it neat**.
* Make and run a query that selects all columns from the event table.  Only return the first 5 rows.
* Use the `sql_head()` function you created to get the first five rows of the sales table.

In [6]:
# Getting table names
get_table_names()

[('users',),
 ('category',),
 ('date',),
 ('listing',),
 ('venue',),
 ('event',),
 ('sales',)]

In [7]:
# Getting column names
user_col_names = get_column_names('users')
category_col_names = get_column_names('category')
date_col_names = get_column_names('date')
listing_col_names = get_column_names('listing')
venue_col_names = get_column_names('venue')
event_col_names = get_column_names('event')
sales_col_names = get_column_names('sales')

In [8]:
# Could also just use a list comprehension vs a bunch of print statements :)
names = get_table_names()
[get_column_names(table_name= x) for x in names]

[[('user_id',),
  ('user_name',),
  ('first_name',),
  ('last_name',),
  ('city',),
  ('state',),
  ('email',),
  ('phone',),
  ('like_sports',),
  ('like_theatre',),
  ('like_concerts',),
  ('like_jazz',),
  ('like_classical',),
  ('like_opera',),
  ('like_rock',),
  ('like_vegas',),
  ('like_broadway',),
  ('like_musicals',)],
 [('cat_id',), ('cat_group',), ('cat_name',), ('cat_desc',)],
 [('date_id',),
  ('cal_date',),
  ('day',),
  ('week',),
  ('month',),
  ('qtr',),
  ('year',),
  ('holiday',)],
 [('list_id',),
  ('seller_id',),
  ('event_id',),
  ('date_id',),
  ('num_tickets',),
  ('price_per_ticket',),
  ('total_price',),
  ('list_time',)],
 [('venue_id',),
  ('venue_name',),
  ('venue_city',),
  ('venue_state',),
  ('venue_seats',)],
 [('event_id',),
  ('venue_id',),
  ('cat_id',),
  ('date_id',),
  ('event_name',),
  ('start_time',)],
 [('sales_id',),
  ('list_id',),
  ('seller_id',),
  ('buyer_id',),
  ('event_id',),
  ('date_id',),
  ('qty_sold',),
  ('price_paid',),
  ('c

In [9]:
# Query on events
sq = """ SELECT * FROM event"""
run_query(sq)

(['event_id', 'venue_id', 'cat_id', 'date_id', 'event_name', 'start_time'],
 [(2, 306, 8, 2114, 'Boris Godunov', datetime.datetime(2008, 10, 15, 20, 0)),
  (3, 302, 8, 1935, 'Salome', datetime.datetime(2008, 4, 19, 14, 30)),
  (4,
   309,
   8,
   2090,
   'La Cenerentola (Cinderella)',
   datetime.datetime(2008, 9, 21, 14, 30)),
  (5, 302, 8, 1982, 'Il Trovatore', datetime.datetime(2008, 6, 5, 19, 0)),
  (6,
   308,
   8,
   2109,
   'L Elisir d Amore',
   datetime.datetime(2008, 10, 10, 19, 30)),
  (7, 309, 8, 1891, 'Doctor Atomic', datetime.datetime(2008, 3, 6, 14, 0)),
  (8, 302, 8, 1832, 'The Magic Flute', datetime.datetime(2008, 1, 6, 20, 0)),
  (9, 308, 8, 2087, 'The Fly', datetime.datetime(2008, 9, 18, 19, 30)),
  (10, 305, 8, 2079, 'Rigoletto', datetime.datetime(2008, 9, 10, 15, 0)),
  (11, 302, 8, 1952, 'Doctor Atomic', datetime.datetime(2008, 5, 1, 19, 30)),
  (12, 303, 8, 2111, 'Ring Cycle', datetime.datetime(2008, 10, 12, 15, 0)),
  (13,
   301,
   8,
   2154,
   'Lucia di

In [10]:
# Use sql_head to get the head of sales
sql_head('sales')

,sales_id,list_id,seller_id,buyer_id,event_id,date_id,qty_sold,price_paid,commission,sale_time
0,2,4,8117,11498,4337,1983,2,76,11.4,6/6/2008 05:00:16
1,3,5,1616,17433,8647,1983,2,350,52.5,6/6/2008 08:26:17
2,4,5,1616,19715,8647,1986,1,175,26.25,6/9/2008 08:38:52
3,5,6,47402,14115,8240,2069,2,154,23.1,8/31/2008 09:17:02
4,6,10,24858,24888,3375,2023,2,394,59.1,7/16/2008 11:59:24


## WHERE - 2 points

Now let's do a bit of filtering with WHERE.  Write and run queries to get the following results.  LIMIT all returns to first five rows. 

* Get venues with >= 10000 seats from the venues table
* Get venues in Arizona
* Get users who have a first name that starts with H
* Get just email addresses of users who gave a .edu email address




In [11]:
# Get big venues... so those with >= than 10000 seats
sq = """ SELECT * FROM venue WHERE venue_seats >= 10000 LIMIT 5;"""
run_query(sq)

(['venue_id', 'venue_name', 'venue_city', 'venue_state', 'venue_seats'],
 [(5, 'Gillette Stadium', 'Foxborough', 'MA', 68756),
  (6, 'New York Giants Stadium', 'East Rutherford', 'NJ', 80242),
  (15, 'McAfee Coliseum', 'Oakland', 'CA', 63026),
  (18, 'Madison Square Garden', 'New York City', 'NY', 20000),
  (67, 'Ralph Wilson Stadium', 'Orchard Park', 'NY', 73967)])

In [12]:
# Get venues in AZ
sq = """ SELECT * FROM venue WHERE venue_state = 'AZ' LIMIT 5;"""
run_query(sq)

(['venue_id', 'venue_name', 'venue_city', 'venue_state', 'venue_seats'],
 [(38, 'US Airways Center', 'Phoenix', 'AZ', 0),
  (65, 'Jobing.com Arena', 'Glendale', 'AZ', 0),
  (92, 'University of Phoenix Stadium', 'Glendale', 'AZ', 0),
  (117, 'Chase Field', 'Phoenix', 'AZ', 0)])

In [13]:
# Get user names of who have names starting with 'H'
sq = """ SELECT * FROM users WHERE user_name LIKE 'H%' LIMIT 5;"""
run_query(sq)

(['user_id',
  'user_name',
  'first_name',
  'last_name',
  'city',
  'state',
  'email',
  'phone',
  'like_sports',
  'like_theatre',
  'like_concerts',
  'like_jazz',
  'like_classical',
  'like_opera',
  'like_rock',
  'like_vegas',
  'like_broadway',
  'like_musicals'],
 [(23,
   'HOH86KXG',
   'Vielka',
   'Glass',
   'Fort Worth',
   'MB',
   'non.ante.bibendum@porttitortellus.org',
   '(170) 580-0881',
   None,
   None,
   None,
   None,
   None,
   None,
   'True',
   None,
   None,
   None),
  (29,
   'HUH27PKK',
   'Helen',
   'Avery',
   'Garland',
   'PE',
   'in.faucibus.orci@ultrices.edu',
   '(385) 925-3875',
   None,
   'False',
   None,
   'True',
   None,
   None,
   None,
   'True',
   'False',
   None),
  (30,
   'HMF84NKS',
   'Guy',
   'Cochran',
   'Effingham',
   'NU',
   'Nullam.enim@Maecenas.com',
   '(757) 207-0052',
   None,
   'True',
   None,
   None,
   'True',
   None,
   None,
   None,
   None,
   None),
  (92,
   'HYP36WEQ',
   'Brittany',
   'Crane'

In [14]:
# Get all .edu email addresses... just the email addresses
sq = """ SELECT email FROM users WHERE email LIKE '%.edu' LIMIT 5;"""
run_query(sq)

(['email'],
 [('Suspendisse.tristique@nonnisiAenean.edu',),
  ('ullamcorper.nisl@Cras.edu',),
  ('vel.est@velitegestas.edu',),
  ('justo.nec.ante@quismassa.edu',),
  ('non.justo.Proin@ametconsectetuer.edu',)])

## GROUP BY and HAVING - 2 points
 
Time to practice some GROUP BY and HAVING operations.  Please write and run queries that do the following:
 
GROUP BY application
* Find the top five venues that hosted the most events.  Alias the count of events as 'events_hosted'.  Also return the venue ID
* Get the number of events hosted in each month.  You'll need to use `date_part()` in your select to select just the months.  Alias this as 'month' and then the count of the number of events hosted as 'events_hosted'. 
* Get the top five sellers who made the most commission.  Alias their total commission made as 'total_com'.  Also get their average commission made and alias as 'avg_com'.  Be sure to also display the seller_id.  
 
HAVING application
* Using the same query as the last one, instead of getting the top five sellers get all sellers who have made a total commission greater than 4000.
* Using the same query as the first groupby, instead of returning the top five venues, return just the ID's of venues that have had greater than 60 events. 

In [15]:
# Which venue_id hosted the most events?  
sq = """ SELECT venue_id, COUNT(venue_id) as events_hosted FROM event GROUP BY venue_id ORDER BY events_hosted DESC LIMIT 5"""
run_query(sq)

(['venue_id', 'events_hosted'],
 [(220, 81), (217, 81), (203, 80), (222, 74), (216, 72)])

In [16]:
# Get the number of events hosted each month
sq = """ SELECT date_part('month', start_time) as month, COUNT(date_id) FROM event GROUP BY month ORDER BY month;"""
run_query(sq)

(['month', 'count'],
 [(1.0, 777),
  (2.0, 711),
  (3.0, 753),
  (4.0, 725),
  (5.0, 727),
  (6.0, 709),
  (7.0, 729),
  (8.0, 737),
  (9.0, 746),
  (10.0, 735),
  (11.0, 726),
  (12.0, 722)])

In [17]:
# Get the top five sellers who made the most commission.  Also display their ID and their average commission
sq = """ SELECT seller_id, SUM(commission) as total_com , AVG(commission) as avg_com FROM sales GROUP BY seller_id ORDER BY total_com DESC LIMIT 5;"""
run_query(sq)

(['seller_id', 'total_com', 'avg_com'],
 [(1140, 4859.85, 347.13214285714287),
  (43551, 4704.75, 470.475),
  (13385, 4274.25, 388.5681818181818),
  (25433, 4147.95, 518.49375),
  (2372, 4073.8500000000004, 678.975)])

In [18]:
# Get the seller id, total commission made, and average commision made for sellers who have made more than $4000
sq = """ SELECT seller_id, SUM(commission) as total_com, AVG(commission) as avg_com FROM 
sales GROUP BY seller_id HAVING SUM(commission) > 4000  ;"""
run_query(sq)

(['seller_id', 'total_com', 'avg_com'],
 [(1140, 4859.8499999999985, 347.13214285714275),
  (2372, 4073.8499999999995, 678.9749999999999),
  (13385, 4274.25, 388.5681818181818),
  (25433, 4147.95, 518.49375),
  (43551, 4704.75, 470.475)])

In [19]:
# Get the ids of venues that have hosted over 60 events. Return just the venue_id
sq = """ SELECT venue_id FROM event GROUP BY venue_id HAVING COUNT(venue_id) > 60;"""
run_query(sq)

(['venue_id'],
 [(237,),
  (238,),
  (239,),
  (226,),
  (209,),
  (205,),
  (201,),
  (216,),
  (220,),
  (221,),
  (207,),
  (243,),
  (222,),
  (248,),
  (208,),
  (217,),
  (218,),
  (215,),
  (203,),
  (228,)])

## JOIN - 2 points
 
Time for some joins.  You've probably noticed by now that there is at least one relational key in each table, but some have more.  For example, sales has a unique sale id, listing id, seller id, buyer id, date id.  This allows you to link each sale to relevant information in other tables.  
 
Please write queries to do the following items:
 
* Join information of users to each sale made.  
* Join information about each venue to each event. 

In [75]:
# Join user information to sales to each user
sq = """ SELECT * FROM sales  LEFT JOIN users ON sales.buyer_id = users.user_id;"""
run_query(sq)

(['sales_id',
  'list_id',
  'seller_id',
  'buyer_id',
  'event_id',
  'date_id',
  'qty_sold',
  'price_paid',
  'commission',
  'sale_time',
  'user_id',
  'user_name',
  'first_name',
  'last_name',
  'city',
  'state',
  'email',
  'phone',
  'like_sports',
  'like_theatre',
  'like_concerts',
  'like_jazz',
  'like_classical',
  'like_opera',
  'like_rock',
  'like_vegas',
  'like_broadway',
  'like_musicals'],
 [(4,
   5,
   1616,
   19715,
   8647,
   1986,
   1,
   175,
   26.25,
   '6/9/2008 08:38:52',
   19715,
   'PZY80KEH',
   'Alexander',
   'Spencer',
   'Jordan Valley',
   'ID',
   'amet@est.edu',
   '(963) 349-0806',
   'False',
   'False',
   None,
   'False',
   None,
   None,
   None,
   None,
   'False',
   None),
  (6,
   10,
   24858,
   24888,
   3375,
   2023,
   2,
   394,
   59.1,
   '7/16/2008 11:59:24',
   24888,
   'QDE27MBM',
   'Simon',
   'Battle',
   'Lockport',
   'NH',
   'fermentum.arcu@sitametante.org',
   '(512) 901-0259',
   None,
   None,
   'Tr

In [21]:
# For each event attach the venue information
sq = """ SELECT * FROM event LEFT JOIN venue ON event.venue_id = venue.venue_id;"""
run_query(sq)

(['event_id',
  'venue_id',
  'cat_id',
  'date_id',
  'event_name',
  'start_time',
  'venue_id',
  'venue_name',
  'venue_city',
  'venue_state',
  'venue_seats'],
 [(2,
   306,
   8,
   2114,
   'Boris Godunov',
   datetime.datetime(2008, 10, 15, 20, 0),
   306,
   'Lyric Opera House',
   'Baltimore',
   'MD',
   0),
  (3,
   302,
   8,
   1935,
   'Salome',
   datetime.datetime(2008, 4, 19, 14, 30),
   302,
   'Detroit Opera House',
   'Detroit',
   'MI',
   0),
  (4,
   309,
   8,
   2090,
   'La Cenerentola (Cinderella)',
   datetime.datetime(2008, 9, 21, 14, 30),
   309,
   'Los Angeles Opera',
   'Los Angeles',
   'CA',
   0),
  (5,
   302,
   8,
   1982,
   'Il Trovatore',
   datetime.datetime(2008, 6, 5, 19, 0),
   302,
   'Detroit Opera House',
   'Detroit',
   'MI',
   0),
  (6,
   308,
   8,
   2109,
   'L Elisir d Amore',
   datetime.datetime(2008, 10, 10, 19, 30),
   308,
   'Metropolitan Opera',
   'New York City',
   'NY',
   0),
  (7,
   309,
   8,
   1891,
   'Doctor

## Subqueries - 2 points

To wrap up let's do several subqueries. Please do the following:

* Get all purchases made by users of live in Arizona
* Get event information for all events that took place in a venue where the venue name ends with 'Stadium'. 
* Get event information for all events where the total ticket sales were greater than $50,000.  

In [81]:
# Get all purchases from users who live in Arizona
sq = """ SELECT event_name FROM event WHERE venue_id IN (SELECT venue_id FROM venue WHERE venue_state IN (SELECT state FROM users WHERE state = 'AZ'));"""
run_query(sq)

(['event_name'],
 [('Crue Fest',),
  ('Eddie Vedder',),
  ('Ringo Starr',),
  ('Ray Lamontagne',),
  ('Ted Nugent',),
  ('Nas',),
  ('Black Eyed Peas',),
  ('The Rowdy Frynds Tour',),
  ('The Kooks',),
  ('Seal',),
  ('Hall and Oates',),
  ('Bow Wow',),
  ('Zappa Plays Zappa',),
  ('Fergie',),
  ('Mark Knopfler',),
  ('Tom Jones',),
  ('Jimmy Eat World',),
  ('Ben E. King',),
  ('El Tri',),
  ('Blue Man Group How To Be A Megastar Tour',),
  ('Michael Buble',),
  ('Foghat',),
  ('Vicente Fernandez',),
  ('Hot Chip',),
  ('Sammy Hagar',),
  ('Rusted Root',),
  ('Smokey Robinson',),
  ('Duffy',),
  ('Sevendust',),
  ('Queensryche',),
  ('Jesse Lacey',),
  ('War',),
  ('The Roots',),
  ('Ted Nugent',),
  ('Jamboree in the Hills',),
  ('Isaac Hayes',),
  ('War',),
  ('Lonestar',),
  ('Lucinda Williams',),
  ('Rusted Root',),
  ('Carrie Underwood',),
  ('Kanye West',),
  ('Widespread Panic',),
  ('The Rowdy Frynds Tour',),
  ('Paramore',),
  ('Styx',),
  ('Mark Knopfler',),
  ('Bootsy Collin

In [41]:
# Get event information for all events that took place in a venue where the name ended in 'Stadium'
sq = """ SELECT * FROM event WHERE venue_id IN (SELECT venue_id FROM venue WHERE venue_name LIKE '%Stadium');"""
run_query(sq)
#get_table_names()

(['event_id', 'venue_id', 'cat_id', 'date_id', 'event_name', 'start_time'],
 [(3803,
   2,
   9,
   2181,
   'Dropkick Murphys',
   datetime.datetime(2008, 12, 21, 14, 0)),
  (3816, 11, 9, 2139, 'Keb Mo', datetime.datetime(2008, 11, 9, 19, 0)),
  (3821,
   79,
   9,
   1885,
   'Charlie Daniels Band',
   datetime.datetime(2008, 2, 28, 19, 30)),
  (3824, 98, 9, 1885, 'Govt Mule', datetime.datetime(2008, 2, 28, 14, 0)),
  (3835, 74, 9, 2073, 'LeAnn Rimes', datetime.datetime(2008, 9, 4, 14, 30)),
  (3837, 67, 9, 1850, 'Randy Travis', datetime.datetime(2008, 1, 24, 15, 0)),
  (3840, 14, 9, 1876, 'Gwen Stefani', datetime.datetime(2008, 2, 19, 19, 30)),
  (3843, 75, 9, 1986, 'Blake Shelton', datetime.datetime(2008, 6, 9, 14, 30)),
  (3854, 6, 9, 1982, 'Gwen Stefani', datetime.datetime(2008, 6, 5, 15, 0)),
  (3862, 116, 9, 1855, 'Teddy Geiger', datetime.datetime(2008, 1, 29, 19, 30)),
  (3864, 75, 9, 1992, 'Celine Dion', datetime.datetime(2008, 6, 15, 15, 0)),
  (3866, 72, 9, 1950, 'Cold War 

In [77]:
# Get event information where the total sales for that event were greater than 50000
sq = """ SELECT * FROM event WHERE event_id IN (SELECT event_id FROM sales GROUP BY event_id HAVING SUM(qty_sold) > 50000 ); """
run_query(sq)

(['event_id', 'venue_id', 'cat_id', 'date_id', 'event_name', 'start_time'], [])